# Docking Workflow Notebook
Target: Acetylcholinesterase (PDB: 4EY7) with Donepezil (E20) binding sites.

## 1. Receptor Preparation

### Step 1.1: Receptor Preparation with PDBFixer

In [1]:
from pdbfixer import PDBFixer
from openmm.app import PDBFile
import os
import glob
from subprocess import run, CalledProcessError

input_pdb_filename = "4ey7.pdb"
output_protein_pdb_filename = "4EY7_prepped.pdb"
target_ph_for_hydrogens = 7.4

if not os.path.exists(input_pdb_filename):
    raise FileNotFoundError(f"Input PDB file '{input_pdb_filename}' not found.")

fixer = PDBFixer(filename=input_pdb_filename)
fixer.findNonstandardResidues()
# E20 (Donepezil) and other heteroatoms will be removed by removeHeterogens.
fixer.removeHeterogens(keepWater=False)
fixer.findMissingResidues()
fixer.findMissingAtoms()
if fixer.missingAtoms:
    fixer.addMissingAtoms() 
fixer.addMissingHydrogens(pH=target_ph_for_hydrogens)

with open(output_protein_pdb_filename, 'w') as f:
    PDBFile.writeFile(fixer.topology, fixer.positions, f)


### Step 1.2: Find E20 sites
Use ChimeraX and 4ey7.pdb, and perform following commands:

`select /A:604`

`save e20_A604_for_box.pdb models sel selectedOnly true`

`select clear`

`select /B:605`

`save e20_B605_for_box.pdb models sel selectedOnly true`

### Step 1.3
Receptor PDBQT and Docking Box Generation for Each E20 Site

Site-specific files will be placed into dedicated directories named by site ID.

In [2]:
def prepare_meeko_receptor_for_site(base_protein_pdb, site_ligand_definition_pdb, site_processing_directory, site_common_name_for_meeko_outputs, box_padding_value):
    """
    Prepares receptor PDBQT and Vina box files for a specific site using Meeko,
    placing outputs in the site_processing_directory.
    Returns a dictionary with paths to key files if successful, None otherwise.
    """
    os.makedirs(site_processing_directory, exist_ok=True)
    # Meeko outputs will be like: <site_processing_directory>/<site_common_name_for_meeko_outputs>.pdbqt
    meeko_output_basepath = os.path.join(site_processing_directory, site_common_name_for_meeko_outputs)

    if not os.path.exists(base_protein_pdb):
        print(f"ERROR: Base receptor PDB '{base_protein_pdb}' not found for Meeko processing of site {site_common_name_for_meeko_outputs}.")
        return None
    if not os.path.exists(site_ligand_definition_pdb):
        print(f"ERROR: Site definition PDB '{site_ligand_definition_pdb}' not found for {site_common_name_for_meeko_outputs}.")
        return None

    meeko_command = (
        f"mk_prepare_receptor.py --read_pdb {base_protein_pdb} "
        f"-o {meeko_output_basepath} -p -v "
        f"--box_enveloping {site_ligand_definition_pdb} --padding {box_padding_value}"
    )
    return_code = os.system(meeko_command)

    receptor_pdbqt_path = f"{meeko_output_basepath}.pdbqt"
    box_config_path = f"{meeko_output_basepath}.box.txt"

    if return_code == 0 and os.path.exists(receptor_pdbqt_path) and os.path.exists(box_config_path):
        return {
            "receptor_pdbqt": receptor_pdbqt_path,
            "box_config": box_config_path,
            "site_dir": site_processing_directory, # The main directory for this site's files
            "site_meeko_base": meeko_output_basepath # Base path for Meeko generated files within site_dir
        }
    else:
        print(f"ERROR: Meeko preparation failed for {site_common_name_for_meeko_outputs} (return code {return_code}).")
        return None

default_box_padding = 10

In [3]:
# Site A604 (E20 originally at /A:604)
site_A604_identifier = "A604"
site_A604_ligand_definition_pdb = "e20_A604_for_box.pdb"
site_A604_main_directory = f"{site_A604_identifier}/" # Directory will be "A604/"
site_A604_meeko_output_basename = f"4EY7_{site_A604_identifier}" # e.g., 4EY7_A604
site_A604_preparation_results = prepare_meeko_receptor_for_site(
    output_protein_pdb_filename,
    site_A604_ligand_definition_pdb,
    site_A604_main_directory,
    site_A604_meeko_output_basename,
    default_box_padding
)


Files written:
  A604/4EY7_A604.pdbqt <-- static (i.e., rigid) receptor input file
A604/4EY7_A604.box.txt <-- Vina-style box dimension file
A604/4EY7_A604.box.pdb <-- PDB file to visualize the grid box


In [4]:
# Site B605 (representing E20 originally at /B:605, using user-defined name "B605" for outputs)
site_B605_identifier = "B605" 
site_B605_ligand_definition_pdb = "e20_B605_for_box.pdb" # User specified this filename
site_B605_main_directory = f"{site_B605_identifier}/" # Directory will be "B605/"
site_B605_meeko_output_basename = f"4EY7_{site_B605_identifier}" # e.g., 4EY7_B605
site_B605_preparation_results = prepare_meeko_receptor_for_site(
    output_protein_pdb_filename,
    site_B605_ligand_definition_pdb,
    site_B605_main_directory,
    site_B605_meeko_output_basename,
    default_box_padding
)


Files written:
  B605/4EY7_B605.pdbqt <-- static (i.e., rigid) receptor input file
B605/4EY7_B605.box.txt <-- Vina-style box dimension file
B605/4EY7_B605.box.pdb <-- PDB file to visualize the grid box


## 2. Ligand Preparation
This section prepares a common set of ligands

### Step 2.1: Generate 3D conformers with Scrubber from SMILES

In [10]:
ligands_smiles_file = "ligands.smi"
generated_sdf_file_from_smiles = "mols_initial.sdf" 

os.system(f"scrub.py {ligands_smiles_file} -o {generated_sdf_file_from_smiles}")

local variable 'energy' referenced before assignment
local variable 'energy' referenced before assignment
local variable 'energy' referenced before assignment
local variable 'energy' referenced before assignment
local variable 'energy' referenced before assignment
local variable 'energy' referenced before assignment


Scrub in progress. Here's how things are going:
Input molecules supplied: 100
mols processed: 94, skipped by rdkit: 0, failed: 6
nr isomers (tautomers and acid/base conjugates): 136 (avg. 1.447 per mol)
nr conformers:  136 (avg. 1.000 per isomer, 1.447 per mol)



local variable 'energy' referenced before assignment
local variable 'energy' referenced before assignment
local variable 'energy' referenced before assignment
local variable 'energy' referenced before assignment
local variable 'energy' referenced before assignment
local variable 'energy' referenced before assignment
local variable 'energy' referenced before assignment
local variable 'energy' referenced before assignment
local variable 'energy' referenced before assignment
local variable 'energy' referenced before assignment
local variable 'energy' referenced before assignment


Scrub in progress. Here's how things are going:
Input molecules supplied: 200
mols processed: 183, skipped by rdkit: 0, failed: 17
nr isomers (tautomers and acid/base conjugates): 273 (avg. 1.492 per mol)
nr conformers:  273 (avg. 1.000 per isomer, 1.492 per mol)



local variable 'energy' referenced before assignment
local variable 'energy' referenced before assignment
{'INITIAL_COORDS': 38, 'FIRST_MINIMIZATION': 1, 'FINAL_CHIRAL_BOUNDS': 296, 'CHECK_CHIRAL_CENTERS2': 35}
local variable 'energy' referenced before assignment
local variable 'energy' referenced before assignment
local variable 'energy' referenced before assignment
{'INITIAL_COORDS': 52, 'FINAL_CHIRAL_BOUNDS': 281, 'CHECK_CHIRAL_CENTERS2': 37}


Scrub completed.
Summary of what happened:
Input molecules supplied: 294
mols processed: 269, skipped by rdkit: 0, failed: 25
nr isomers (tautomers and acid/base conjugates): 425 (avg. 1.580 per mol)
nr conformers:  425 (avg. 1.000 per isomer, 1.580 per mol)


{'INITIAL_COORDS': 344, 'CHECK_TETRAHEDRAL_CENTERS': 2, 'FINAL_CHIRAL_BOUNDS': 214}


0

### Step 2.2: Remove salts and keep largest fragment using RDKit

In [15]:
from rdkit import Chem
from rdkit.Chem import SaltRemover, SDWriter

input_sdf_for_rdkit_processing = generated_sdf_file_from_smiles
output_sdf_after_rdkit_processing = "mols_desalted.sdf" 
ligands_written_count_after_rdkit = 0

supplier = Chem.SDMolSupplier(input_sdf_for_rdkit_processing, removeHs=False, sanitize=False)
remover = SaltRemover.SaltRemover()
with SDWriter(output_sdf_after_rdkit_processing) as writer:
    for mol in supplier:
        if mol is None: continue
        Chem.SanitizeMol(mol)
        stripped_mol = remover.StripMol(mol, dontRemoveEverything=True)
        fragments = Chem.GetMolFrags(stripped_mol, asMols=True, sanitizeFrags=True)
        if not fragments: continue
        
        valid_fragments = [f for f in fragments if f is not None]
        if not valid_fragments: continue
        largest_fragment = max(valid_fragments, key=lambda f: f.GetNumHeavyAtoms(), default=None)
        
        if largest_fragment:
            writer.write(largest_fragment)
            ligands_written_count_after_rdkit += 1
      

### Step 2.3: Prepare ligand PDBQT files with Meeko

In [16]:
common_ligand_pdbqt_dir = "common_ligands_pdbqt/" 

os.makedirs(common_ligand_pdbqt_dir, exist_ok=True)
os.system(f"mk_prepare_ligand.py -i {output_sdf_after_rdkit_processing} --multimol_outdir {common_ligand_pdbqt_dir}")


Maybe this molecule is "too" symmetric? CHEMBL89354 CC(C)(C)c1cccc(C(=O)C(F)(F)F)c1


Renaming CHEMBL187818_i0 to CHEMBL187818_i0-again1 to disambiguate filename
Renaming CHEMBL187818_i1 to CHEMBL187818_i1-again1 to disambiguate filename
Renaming CHEMBL187818_i2 to CHEMBL187818_i2-again1 to disambiguate filename
Renaming CHEMBL3585780 to CHEMBL3585780-again1 to disambiguate filename
Renaming CHEMBL426441 to CHEMBL426441-again1 to disambiguate filename
Renaming CHEMBL188823 to CHEMBL188823-again1 to disambiguate filename
Renaming CHEMBL3623551 to CHEMBL3623551-again1 to disambiguate filename
Renaming CHEMBL3585781 to CHEMBL3585781-again1 to disambiguate filename
Renaming CHEMBL3585775 to CHEMBL3585775-again1 to disambiguate filename
Renaming CHEMBL3585784 to CHEMBL3585784-again1 to disambiguate filename
Renaming CHEMBL186264_i0 to CHEMBL186264_i0-again1 to disambiguate filename
Renaming CHEMBL186264_i1 to CHEMBL186264_i1-again1 to disambiguate filename
Renaming CHEMBL186264_i2 to CHEMBL186264_i2-again1 to disambiguate filename
Renaming CHEMBL4876038_i0 to CHEMBL4876038_i

Input molecules processed: 425, skipped: 0
PDBQT files written: 425
PDBQT files not written due to error: 0
Input molecules with errors: 0


Renaming CHEMBL292314_i0 to CHEMBL292314_i0-again5 to disambiguate filename
Renaming CHEMBL292314_i1 to CHEMBL292314_i1-again5 to disambiguate filename
Renaming CHEMBL4864555_i0 to CHEMBL4864555_i0-again1 to disambiguate filename
Renaming CHEMBL4864555_i1 to CHEMBL4864555_i1-again1 to disambiguate filename
Renaming CHEMBL94 to CHEMBL94-again1 to disambiguate filename


0

## 3. Running AutoDock Vina for Each Prepared Site

In [17]:
def run_vina_docking_for_prepared_site(
    site_tag, # e.g., "A604" or "B605"
    site_meeko_data, # This is the dictionary returned by prepare_meeko_receptor_for_site
    common_ligands_pdbqt_directory, 
    vina_executable_cmd, 
    vina_cpu_cores, 
    vina_search_exhaustiveness, 
    vina_number_of_modes
):
    """Runs AutoDock Vina for all ligands against a specific prepared site."""
    
    receptor_pdbqt_file = site_meeko_data["receptor_pdbqt"]
    box_config_file = site_meeko_data["box_config"]
    # site_main_dir is the parent directory for this site (e.g., "A604/")
    site_main_dir = site_meeko_data["site_dir"] 
    
    vina_results_subdir = os.path.join(site_main_dir, "vina_results") # Subdirectory within the site's main directory
    vina_logs_subdir = os.path.join(site_main_dir, "vina_logs") 
    os.makedirs(vina_results_subdir, exist_ok=True)
    os.makedirs(vina_logs_subdir, exist_ok=True)

    if not (os.path.exists(receptor_pdbqt_file) and os.path.exists(box_config_file)):
        return

    all_ligands_for_docking = glob.glob(os.path.join(common_ligands_pdbqt_directory, "*.pdbqt"))
    if not all_ligands_for_docking:
        return
    
    print(f"\n--- Starting Vina Docking for Site: {site_tag} (Files in {site_main_dir}) ---")

    for ligand_pdbqt_filepath in all_ligands_for_docking:
        ligand_filename_base = os.path.basename(ligand_pdbqt_filepath).replace(".pdbqt", "")
        # Output filenames are simpler as they are already in site-specific directories
        docked_poses_output_filepath = os.path.join(vina_results_subdir, f"{ligand_filename_base}_docked.pdbqt")
        vina_captured_output_log_filepath = os.path.join(vina_logs_subdir, f"{ligand_filename_base}_vina_log.txt") 
        
        vina_execution_command = [
            vina_executable_cmd,
            "--receptor", receptor_pdbqt_file,
            "--ligand", ligand_pdbqt_filepath,
            "--config", box_config_file,
            "--out", docked_poses_output_filepath,
            "--cpu", str(vina_cpu_cores),
            "--exhaustiveness", str(vina_search_exhaustiveness),
            "--num_modes", str(vina_number_of_modes)
        ]
        
        try:
            vina_run_result = run(vina_execution_command, capture_output=True, text=True, check=True)
            with open(vina_captured_output_log_filepath, 'w') as f_log_output:
                f_log_output.write("--- Vina Standard Output ---\n")
                f_log_output.write(vina_run_result.stdout)
                f_log_output.write("\n--- Vina Standard Error ---\n")
                f_log_output.write(vina_run_result.stderr)
        except CalledProcessError as e_vina_run:
            with open(vina_captured_output_log_filepath, 'w') as f_log_error_output:
                f_log_error_output.write(f"Vina command failed with return code {e_vina_run.returncode}\n")
                if e_vina_run.stdout: f_log_error_output.write(f"--- Vina STDOUT ---\n{e_vina_run.stdout}\n")
                if e_vina_run.stderr: f_log_error_output.write(f"--- Vina STDERR ---\n{e_vina_run.stderr}\n")
        except FileNotFoundError:
            print(f"ERROR: Vina executable '{vina_executable_cmd}' not found. Halting docking for site {site_tag}.")
            return 
        except Exception: 
            pass 

# Vina General Configuration
vina_executable_command = "vina" 
vina_cpu_setting_val = "0"
vina_exhaustiveness_setting_val = "8"
vina_num_modes_setting_val = "9"

In [18]:
# Docking for Site A604
if site_A604_preparation_results:
    run_vina_docking_for_prepared_site(
        site_tag=site_A604_identifier, 
        site_meeko_data=site_A604_preparation_results,
        common_ligands_pdbqt_directory=common_ligand_pdbqt_dir,
        vina_executable_cmd=vina_executable_command,
        vina_cpu_cores=vina_cpu_setting_val,
        vina_search_exhaustiveness=vina_exhaustiveness_setting_val,
        vina_number_of_modes=vina_num_modes_setting_val
    )
else:
    print(f"Skipping Vina docking for site {site_A604_identifier} as its preparation failed or was skipped.")


--- Starting Vina Docking for Site: A604 (Files in A604/) ---


In [11]:
# Docking for Site B605
if site_B605_preparation_results:
    run_vina_docking_for_prepared_site(
        site_tag=site_B605_identifier, 
        site_meeko_data=site_B605_preparation_results,
        common_ligands_pdbqt_directory=common_ligand_pdbqt_dir,
        vina_executable_cmd=vina_executable_command,
        vina_cpu_cores=vina_cpu_setting_val,
        vina_search_exhaustiveness=vina_exhaustiveness_setting_val,
        vina_number_of_modes=vina_num_modes_setting_val
    )
else:
    print(f"Skipping Vina docking for site {site_B605_identifier} as its preparation failed or was skipped.")


--- Starting Vina Docking for Site: B605 (Files in B605/) ---


In [23]:
import glob, os, pandas as pd

def parse_vina_log(f):
    try:
        L = open(f).read().splitlines()
        i = next((i for i,l in enumerate(L) if l.startswith("mode |   affinity")), None)
        j = next((j for j in range(i+1,len(L)) if L[j].startswith("-----")), None) if i is not None else None
        if j is None: return None
        for p in (l.split() for l in L[j+1:]):
            if p and p[0]=="1": return float(p[1])
            if not p or not p[0].isdigit(): break
    except:
        return None

def rank_site(id, dir, top=10):
    logs = glob.glob(os.path.join(dir, "vina_logs", "*_vina_log.txt"))
    if not logs:
        raise FileNotFoundError(f"No Vina log files in {dir}")
    df = pd.DataFrame([
        {"Ligand": os.path.basename(l).replace("_vina_log.txt",""),
         "Affinity": a, "Site": id}
        for l in logs if (a := parse_vina_log(l)) is not None
    ])
    if df.empty:
        raise ValueError(f"No valid affinities in {dir}")
    df = df.sort_values("Affinity")
    print(f"\n--- Top {top} Ligands for Site: {id} ---\n{df.head(top).to_string(index=False)}")
    return df

all_dfs = []
for id, prep, d in [
    (site_A604_identifier, site_A604_preparation_results, site_A604_main_directory),
    (site_B605_identifier, site_B605_preparation_results, site_B605_main_directory)
]:
    if prep:
        try:
            all_dfs.append(rank_site(id, d, 10))
        except Exception as e:
            print(f"Error ranking site {id}: {e}")

if all_dfs:
    combined = pd.concat(all_dfs, ignore_index=True).sort_values("Affinity")
    print("\n--- Combined Top Ligands (Sorted by Affinity) ---")
    print(combined.head(len(combined)).to_string(index=False))
else:
    print("\nNo ligand affinities successfully parsed and ranked for any site.")



--- Top 10 Ligands for Site: A604 ---
          Ligand  Affinity Site
   CHEMBL4214707    -14.06 A604
   CHEMBL4218191    -13.55 A604
CHEMBL5423918_i2    -13.51 A604
   CHEMBL4210316    -13.43 A604
CHEMBL3356537_i0    -13.41 A604
   CHEMBL1084256    -13.38 A604
   CHEMBL4209803    -13.35 A604
    CHEMBL140999    -13.29 A604
   CHEMBL4213792    -13.27 A604
CHEMBL5423918_i0    -13.20 A604

--- Top 10 Ligands for Site: B605 ---
          Ligand  Affinity Site
   CHEMBL4214707    -14.11 B605
CHEMBL3356537_i1    -13.24 B605
   CHEMBL4205144    -13.22 B605
CHEMBL4159705_i0    -13.21 B605
CHEMBL3356536_i4    -13.21 B605
   CHEMBL4213792    -13.20 B605
CHEMBL3356537_i2    -13.18 B605
CHEMBL5423918_i9    -13.18 B605
   CHEMBL1084256    -13.16 B605
CHEMBL3356537_i0    -13.10 B605

--- Combined Top Ligands (Sorted by Affinity) ---
                 Ligand  Affinity Site
          CHEMBL4214707   -14.110 B605
          CHEMBL4214707   -14.060 A604
          CHEMBL4218191   -13.550 A604
       CHEM

--- Combined Top 20 Ligands (Sorted by Affinity) ---
                 Ligand  Affinity Site
          CHEMBL4214707   -14.110 B605
          CHEMBL4214707   -14.060 A604
          CHEMBL4218191   -13.550 A604
       CHEMBL5423918_i2   -13.510 A604
          CHEMBL4210316   -13.430 A604
       CHEMBL3356537_i0   -13.410 A604
          CHEMBL1084256   -13.380 A604
          CHEMBL4209803   -13.350 A604
           CHEMBL140999   -13.290 A604
          CHEMBL4213792   -13.270 A604
       CHEMBL3356537_i1   -13.240 B605
          CHEMBL4205144   -13.220 B605
       CHEMBL4159705_i0   -13.210 B605
       CHEMBL3356536_i4   -13.210 B605
          CHEMBL4213792   -13.200 B605
       CHEMBL5423918_i0   -13.200 A604
       CHEMBL5423918_i1   -13.190 A604
       CHEMBL5423918_i9   -13.180 B605
       CHEMBL3356537_i2   -13.180 B605
       CHEMBL1651129_i0   -13.180 A604